In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# 


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Data Import

In [ ]:
sample_solution_df = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
train_df = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
test_df = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
sample_solution_df

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
#shape Check
print("train/test:{}/{}".format(train_df.shape,test_df.shape))

In [ ]:
train_df.dtypes.unique()
test_df.dtypes.unique()

In [ ]:
#train, testともに
train_col_l = set(train_df.columns)
test_col_l = set(test_df.columns)
col_diff = train_col_l ^ test_col_l
col_diff

# 2. Check NAN in the data frame

In [ ]:
# train_df.isnull
train_df.isnull().sum().sort_values(ascending=False).plot.bar()

In [ ]:
test_df.isnull().sum().sort_values(ascending=False).plot.bar()

In [ ]:
# train_X = train_df[~train_df["claim"]]
train_X = train_df.drop("claim", axis=1).reset_index(drop=True)
train_y = train_df["claim"].reset_index(drop=True)
test_X = test_df.reset_index(drop=True)
train_y

In [ ]:
train_df["claim"]

# 3.Check Base line as Light GBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
folds=3
kf = KFold(n_splits=folds)

lgb_params = {"objective":"binary","random_seed":1234}
#adjust the hyper parameters
# lgb_params = {"objective":"binary", "max_bin":331, "num_leaves":20, "min_data_in_leaf":57, "random_seed":1234}

In [ ]:
train_y.unique()

In [ ]:
models=[]
for train_index, valid_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    y_train = train_y.iloc[train_index]
    X_valid = train_X.iloc[valid_index]
    y_valid = train_y.iloc[valid_index]
#     print(len(X_train.columns),len(y_train.columns),len(X_valid.columns),len(y_valid.columns))
    
    lgb_train= lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    
    model_lgb = lgb.train(lgb_params, lgb_train, valid_sets=lgb_valid, num_boost_round=100, early_stopping_rounds=20, verbose_eval=10)
    
    y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
    print(accuracy_score(np.round(y_pred), y_valid))
    
    models.append(model_lgb)

In [ ]:
for model in models:
    lgb.plot_importance(model, importance_type="gain", max_num_features=15)

In [ ]:
preds=[]
for model in models:
    pred = model_lgb.predict(test_X)
    preds.append(pred)

In [ ]:
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis=0)

In [ ]:
preds_int = (preds_mean > 0.5).astype(int)

In [ ]:
solution_df = sample_solution_df.copy()
solution_df["claim"]= preds_int

In [ ]:
# os.mkdir("../submit")
# solution_df.to_csv("./TabularPlayGroundSeries_Sep2021_rev00.csv", index=False)